In [40]:
import warnings
warnings.filterwarnings('ignore')

from pymongo import MongoClient
import pprint

import copy
import pandas as pd

# Requests sends and recieves HTTP requests.
import requests 
# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup as soup

from urllib.request import urlopen as uReq
from collections import OrderedDict

###############

In [2]:
my_url = 'https://www.makeupalley.com/product/searching?Brand=0&BrandName=&CategoryID=7&q='

In [7]:
#opening connection, download the webpage
uClient = uReq(my_url)
#get html
page_html = uClient.read()
#close the request
uClient.close()

In [8]:
request(my_url)

In [9]:
#html parser
page_soup = soup(page_html, "html.parser")

In [10]:
page_soup.h1

<h1><a href="/" routerlink="/"><img alt="MakeupAlley" class="mua-logo-pic" src="../../../img/logo/logo-mobile.svg"/></a></h1>

In [11]:
page_soup.p

<p class="results-info"> Displaying results 1-15 of 24,733 results found </p>

In [12]:
links = page_soup.findAll('a',{'class':'item-name'})

In [13]:
len(links)

30

In [14]:
#use OrderedDict to preserve the order
links = list(OrderedDict.fromkeys(links))
len(links)

15

In [19]:
links[0]['href']

'/product/showreview.asp/ItemId=206387/Detox-in-a-Jar-Exfoliating-Face-Mask/Tula/Masks'

In [20]:
#get 15 product links from each page
prod_url = []
for l in links:
    prod_url.append('https://www.makeupalley.com{}'.format(l['href']))

In [21]:
#get product name
uClient = uReq(prod_url[0])
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, 'html.parser')
product = page_soup.find('div',{'class':'headline'}).get_text()
product

'Detox in a Jar Exfoliating Face MaskTula'

In [24]:
#brand name
uClient = uReq(prod_url[0])
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, 'html.parser')
brand = page_soup.find('span',{'class':'brand'}).get_text()
brand

'Tula'

In [31]:
#User names list
uClient = uReq(prod_url[0])
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, 'html.parser')
user = page_soup.find('div',{'class':'__UserLink__'}).get_text()
# users = []
# for i in range(len(user)):
#     users.append(user[i].get_text())
print(users)

['flairgirl8']


In [32]:
#try reading reviews from first link from first page

uClient = uReq(prod_url[0])
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, 'html.parser')
review = page_soup.find('span',{'class': 'review-text'}).get_text()
# reviews = []
# for i in range(len(review)):
#     reviews.append(review[i].get_text())
print(review)


This is a light blue clay-based mask. It has a light, powdery scent. Easy to use. It contains lactic acid to gently exfoliate skin and also has a small amount of scrubbing beads in it. I left this on my skin for 10 minutes and washed off. My skin looked and felt smoother and softer. 


In [36]:
##need to figure out how to find out the total pages for each product review
# for i in range(1,5):
#     for j in range(1,5):
#         print(prod_url[i]+'?page={}#reviews'.format(j))

https://www.makeupalley.com/product/showreview.asp/ItemId=74011/Vitamin-E-Skin-Care-Cream/Fruit-Of-The-Earth/Moisturizers?page=1#reviews
https://www.makeupalley.com/product/showreview.asp/ItemId=74011/Vitamin-E-Skin-Care-Cream/Fruit-Of-The-Earth/Moisturizers?page=2#reviews
https://www.makeupalley.com/product/showreview.asp/ItemId=74011/Vitamin-E-Skin-Care-Cream/Fruit-Of-The-Earth/Moisturizers?page=3#reviews
https://www.makeupalley.com/product/showreview.asp/ItemId=74011/Vitamin-E-Skin-Care-Cream/Fruit-Of-The-Earth/Moisturizers?page=4#reviews
https://www.makeupalley.com/product/showreview.asp/ItemId=197848/Kora-Organics-Noni-Glow-Face-Oil/KORA-Organics/Treatments-(Face)?page=1#reviews
https://www.makeupalley.com/product/showreview.asp/ItemId=197848/Kora-Organics-Noni-Glow-Face-Oil/KORA-Organics/Treatments-(Face)?page=2#reviews
https://www.makeupalley.com/product/showreview.asp/ItemId=197848/Kora-Organics-Noni-Glow-Face-Oil/KORA-Organics/Treatments-(Face)?page=3#reviews
https://www.makeu

**Create some functions to iterate through all pages**

In [37]:
#get url for every page 
pages_url = []
for i in range(1,1650):
    pages_url.append('https://www.makeupalley.com/product/searching?Brand=0&BrandName=&CategoryID=7&q=&page={}'.format(i))

pages_url[0:2]

['https://www.makeupalley.com/product/searching?Brand=0&BrandName=&CategoryID=7&q=&page=1',
 'https://www.makeupalley.com/product/searching?Brand=0&BrandName=&CategoryID=7&q=&page=2']

In [50]:
#store pages link in text file
def get_page_url(page_count):
    for i in range(1,page_count+1):
        with open(file='pages_url.txt',mode='a',newline='') as fd:
            fd.write('https://www.makeupalley.com/product/searching?Brand=0&BrandName=&CategoryID=7&q=&page={}'.format(i))

In [51]:
#get_page_url(1650)

In [55]:
#url = pd.read_csv('pages_url.txt', header= None)

In [56]:
#url.head()

,0
0,https://www.makeupalley.com/product/searching?...
1,https://www.makeupalley.com/product/searching?...
2,https://www.makeupalley.com/product/searching?...
3,ttps://www.makeupalley.com/product/searching?B...
4,https://www.makeupalley.com/product/searching?...


In [39]:
#get all product links from all pages
for l in pages_url:
    uClient = uReq(l)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, 'html.parser')
    prod_links = page_soup.findAll('a',{'class':'item-name'})
    prod_links = list(OrderedDict.fromkeys(links))
    
    prod_url = []
    for _ in range(len(pages_url)):
        for l in prod_links:
            prod_url.append('https://www.makeupalley.com{}'.format(l['href']))
print(len(prod_url))
print(prod_url[0:3])

24735
['https://www.makeupalley.com/product/showreview.asp/ItemId=206387/Detox-in-a-Jar-Exfoliating-Face-Mask/Tula/Masks', 'https://www.makeupalley.com/product/showreview.asp/ItemId=74011/Vitamin-E-Skin-Care-Cream/Fruit-Of-The-Earth/Moisturizers', 'https://www.makeupalley.com/product/showreview.asp/ItemId=197848/Kora-Organics-Noni-Glow-Face-Oil/KORA-Organics/Treatments-(Face)']


In [ ]:
#get all product links from all pages & store in txt file
def get_product_url():
    for l in pages_url:
        uClient = uReq(l)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, 'html.parser')
        prod_links = page_soup.findAll('a',{'class':'item-name'})
        prod_links = list(OrderedDict.fromkeys(links))

        prod_url = []
        for _ in range(len(pages_url)):
            for l in prod_links:
                prod_url.append('https://www.makeupalley.com{}'.format(l['href']))

In [ ]:
#in each product url, need to get all the reviews from all pages
#NEED TO KNOW HOW TO GO TO THE NEXT PAGE

In [ ]:
#create a row containing product name, brand name, user name, review text, rating to append to a dataframe

def create_row(page_url):
    #15 reviews per page
    for i in range(1,15): 
        uClient = uReq('https://www.makeupalley.com{}'.format(prod_url[2]))
        page_html = uClient.read()
        uClient.close()
    page_soup = soup(page_html, 'html.parser')
    product = page_soup.find('div',{'class':'headline'}).get_text()
    brand = page_soup.find('span',{'class':'brand'}).get_text()
    user = page_soup.find('div',{'class':'__UserLink__'}).get_text()
    review = page_soup.find('span',{'class': 'review-text'}).get_text()
    rating = page_soup.find('span',{'class': 'rating-value'}).get_text()
    row.extend(product, user, review, rating)

In [ ]:
# def clean_string(column):
#     return column.apply(lambda x: x.replace("\n",'',2)).apply(lambda x: x.replace('  ',''))

# def scrape_reviews(PATH, n_pages, sleep_time = 0.3):

#     product = []
#     brand = []
#     user = []
#     review = []
#     rating = []

#     for p in range(n_pages):

#         sleep(sleep_time)

#         http = requests.get(f'{PATH}{p}')
#         bsoup = BeautifulSoup(http.text, 'html.parser')
#         page_soup = soup(page_html, 'html.parser')
        
#         product_containers = page_soup.findAll('div',{'class':'headline'})
#         brand_containers = page_soup.findAll('span',{'class':'brand'})
#         user_containers = page_soup.findAll('div',{'class':'__UserLink__'})
#         review_containers = page_soup.findAll('span',{'class': 'review-text'})
#         rating_container = page_soup.findAll('span',{'class': 'rating-value'})
    

#         for x in range(len(data_containers)):    
#             review_c = review_containers[x]
#             headers.append(review_c.h2.a.text)
#             reviews.append(review_c.p.text)
#             reviewer = user_containers[x]
#             names.append(reviewer.h3.text)
#             rating = rating_container[x]
#             ratings.append(rating.div.attrs['class'][1][12])
            


#             prof = profile_link_containers[x]
#             link = 'https://www.trustpilot.com'+ prof.a['href']
#             c_profile = requests.get(f'{link}')
#             csoup = BeautifulSoup(c_profile.text, 'html.parser')
#             cust_container = csoup.find('div', class_ = 'user-summary-location')
#             locations.append(cust_container.text)
            
#     rev_df = pd.DataFrame(list(zip(product, brand, user, review, rating)),
#                   columns = ['Product','Brand','User','Review','Rating'])
    
#     rev_df.Product = clean_string(rev_df.Product)
#     rev_df.Brand = clean_string(rev_df.Brand)
#     rev_df.User = clean_string(rev_df.User)
#     rev_df.Review = clean_string(rev_df.Review)
#     rev_df.Rating = rev_df.Rating.astype('int')
    
#     return rev_df

In [41]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree
import re
from time import sleep
from sqlalchemy import create_engine
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String, MetaData, Float
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.exc import IntegrityError
from sqlalchemy.sql import select, text

In [ ]:
def make_soup(url):
    headers = {
        'user-agent': ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:32.0) '
                       'Gecko/20100101 Firefox/32.0')}
    response = requests.get(url, headers=headers)
    print('Fetched ', url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    return soup

In [ ]:
def make_ma_review_soup(product_id, page):
    review_url = 'http://reviews.sephora.com/8723abredes/{}/reviews.htm?format=embedded&page={}'.format(
        product_id, page)
    soup = make_soup(review_url)
    return soup

In [ ]:
def scrape_product_reviews(product_id):
    page = 1
    soup = make_sephora_review_soup(product_id, page)
    total_review_count = 0
    number_reviews_tag = soup.find('div', id='BVRRCustomRatingCountId')
    if number_reviews_tag:
        total_review_count = number_reviews_tag.find(
            'span', class_='BVRRNumber').text
        total_review_count = total_review_count.replace(',', '')
        total_review_count = int(total_review_count)

    if total_review_count == 0:
        return None

    curr_review_count = 0
    reviews = []

    while (curr_review_count) < 200 and (
            curr_review_count < total_review_count):
        reviews_grid = soup.find('div', id='BVRRContentContainerID')
        reviews_tag = reviews_grid.findAll('span', {'itemprop': 'review'})
        print('got ', curr_review_count)
        for review_tag in reviews_tag:
            curr_review_count += 1
            age_range = skin_type = skin_tone = eye_color = None
            review_text = reviewer_username = tags = title = None

            review_text = review_tag.find('span', class_='BVRRReviewText').text
            title = review_tag.find('span', class_='BVRRReviewTitle').text
            rating = review_tag.find('span', class_='BVRRRatingNumber').text

            tags_tag = review_tag.find(
                'span', class_='BVRRReviewProTags')
            if tags_tag:
                tags_text = tags_tag.text
                tags_text = tags_text.replace('"', '')
                tags = tags_text.split(',')
                tags = [t.strip() for t in tags]

            username_tag = review_tag.find(
                'span', class_='BVRRNickname')
            if username_tag:
                reviewer_username = username_tag.text.strip()

            skin_type_tag = review_tag.find(
                'span', class_='BVRRContextDataValueskinType')
            if skin_type_tag:
                skin_type = skin_type_tag.text.strip().lower()

            skin_tone_tag = review_tag.find(
                'span', class_='BVRRContextDataValueskinTone')
            if skin_tone_tag:
                skin_tone = skin_tone_tag.text.strip().lower()

            eye_color_tag = review_tag.find(
                'span', class_='BVRRContextDataValueeyeColor')
            if eye_color_tag:
                eye_color = eye_color_tag.text.strip().lower()

            if (review_text is not None):
                review = {'product_id': product_id,
                          'review_text': review_text,
                          'review_title': title,
                          'rating': rating,
                          'age_range': age_range,
                          'skin_type': skin_type,
                          'skin_tone': skin_tone,
                          'eye_color': eye_color,
                          'reviewer_username': reviewer_username,
                          'tags': tags}
                reviews.append(review)
        # Get next review page
        page += 1
        soup = make_sephora_review_soup(product_id, page)

    return reviews